In [1]:
# Import required libraries
import pandas as pd
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px


In [3]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv')

In [4]:
spacex_df.head (5)

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [5]:
locations = spacex_df['Launch Site'].unique()
locations

array(['CCAFS LC-40', 'VAFB SLC-4E', 'KSC LC-39A', 'CCAFS SLC-40'],
      dtype=object)

In [6]:
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [7]:
# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
    dcc.Dropdown(id='site-dropdown',
        options=[{'label': 'All Sites', 'value': 'ALL'},
        {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
        {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
        {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
        {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
        ],
        value='ALL',
        placeholder="Select Launch Site Location",
        searchable=True
    ),
                                
    html.Br(),

    # TASK 2: Add a pie chart to show the total successful launches count for all sites
    # If a specific launch site was selected, show the Success vs. Failed counts for the site
    html.Div(dcc.Graph(id='success-pie-chart')),
    #html.Div(id='success-pie-chart', className='chart-grid', style={'display':'flex', 'flexDirection': 'column'}),
    html.Br(),

    html.P("Payload range (Kg):"),
    # TASK 3: Add a slider to select payload range
    #dcc.RangeSlider(id='payload-slider',...)

    dcc.RangeSlider(id='payload-slider',
                min=0, max=10000, step=1000,
                marks={0: '0',
                       2500: '2500',
                       5000: '5000',
                       7500: '7500',
                       10000: '10000'},
                value=[min_payload, max_payload]
    ),
        
    # TASK 4: Add a scatter chart to show the correlation between payload and launch success
    html.Div(dcc.Graph(id='success-payload-scatter-chart'))

])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output

# Function decorator to specify function input and output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))

def get_pie_chart(entered_site):

    if entered_site == 'ALL':

        # calculation of ALL case results
        da_ba = spacex_df[['class','Launch Site']]
        succ = da_ba[da_ba['class'] == 1]
        succ = succ.groupby(['Launch Site']).count().reset_index()
        
        
        fig = px.pie(values=succ['class'], names=succ['Launch Site'], title='Total Success Launches by Site')
#        fig = px.pie(values=values, title='Total Success Launches for site')
    
    else:

        site = [entered_site]

        da_ba = spacex_df[['class','Launch Site']]
        dat_pay = da_ba[da_ba['Launch Site'] == site[0]]
        dat_pay = dat_pay.groupby(['class']).count().reset_index()
        
        fig = px.pie(values=dat_pay['Launch Site'], names=dat_pay['class'], title='Total Success Launches for site %s' % entered_site)
#        fig = px.pie(values=dat_pay['Launch Site'], names=dat_pay['class'], title='Total Success Launches for site')
#        fig = px.pie(values=values, title='local')
    
    return fig


# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'), 
              Input(component_id="payload-slider", component_property="value"))


def get_sct_chart(entered_site, payload):

    if entered_site == 'ALL':

        fig2 = px.scatter(spacex_df, x="Payload Mass (kg)", y="class", color="Booster Version Category")
        fig2.update_xaxes(range=payload)

    else:

        site = [entered_site]

        dat_sct = spacex_df[['class', 'Launch Site', 'Payload Mass (kg)', 'Booster Version Category']]
        dat_sct = dat_sct[dat_sct['Launch Site'] == site[0]]

        fig2 = px.scatter(dat_sct, x="Payload Mass (kg)", y="class", color="Booster Version Category")
        fig2.update_xaxes(range=payload)

    return fig2


# Run the app
if __name__ == '__main__':
    app.run()


In [16]:
dat_sct = spacex_df[['class', 'Launch Site', 'Payload Mass (kg)', 'Booster Version Category']]
dat_sct.tail(10)

,class,Launch Site,Payload Mass (kg),Booster Version Category
46,1,KSC LC-39A,5200.00,FT
47,1,KSC LC-39A,3500.00,B4
48,1,KSC LC-39A,3600.00,B5
49,1,CCAFS SLC-40,2205.00,FT
50,1,CCAFS SLC-40,3696.65,B4
51,0,CCAFS SLC-40,4230.00,FT
52,0,CCAFS SLC-40,6092.00,B4
53,0,CCAFS SLC-40,2647.00,B4
54,1,CCAFS SLC-40,362.00,B4
55,0,CCAFS SLC-40,5384.00,B4
